In [ ]:
import sys
sys.path.append('../..')
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
import ta
import numpy as np
import datetime
from IPython.display import clear_output
from utilities.data_manager import ExchangeDataManager
from utilities.custom_indicators import get_n_columns, SmoothedHeikinAshi
from utilities.bt_analysis import get_metrics, backtest_analysis
from utilities.plot_analysis import plot_equity_vs_asset, plot_futur_simulations, plot_bar_by_month
import nest_asyncio
nest_asyncio.apply()

In [ ]:
class Strategy():
    def __init__(
        self,
        pair,
        df,
        type=["long"],
        params={},
    ):
        self.pair = pair
        self.df = df
        self.use_long = True if "long" in type else False
        self.use_short = True if "short" in type else False
        self.params = params

        
    def populate_indicators(self):
        params = self.params
        df = self.df
        df.drop(
            columns=df.columns.difference(['open','high','low','close','volume']), 
            inplace=True
        )
        
        # -- Populate indicators --
        df['fast_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["fast_ma"])
        df['slow_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["slow_ma"])
        df['mrat'] = df['fast_ma'] / df['slow_ma']
        df['signal'] = ta.trend.sma_indicator(close=df['mrat'], window=params["signal"])
    
        self.df = df

        return self.df
    
    def populate_buy_sell(self): 
        params = self.params
        df = self.df
        # -- Initiate populate --
        df["close_long"] = False
        df["close_short"] = False
        df["open_short"] = False
        df["open_long"] = False
            
        if self.use_long:
            df.loc[
                (df['mrat'] >= df['signal'])
                , "open_long"
            ] = True
            
            # -- Populate close long --
            df.loc[
                (df['mrat'] < df['signal'])
                , "close_long"
            ] = True
            
        
        if self.use_short:
            df.loc[
                (df['mrat'] < df['signal'])
                , "open_short"
            ] = True
            
            df.loc[
                (df['mrat'] >= df['signal'])
                , "close_short"
            ] = True
                

        self.df = df
        
        return self.df
        
    def run_backtest(self, initial_wallet=1000, leverage=1):
        params = self.params
        df_ini = self.df[:]
        wallet = initial_wallet
        long_exposition = 0
        short_exposition = 0
        maker_fee = 0.0002
        taker_fee = 0.0006
        trades = []
        days = []
        current_day = 0
        previous_day = 0
        current_position = None
        
        for index, row in df_ini.iterrows():
            # -- Add daily report --
            current_day = index.day
            if previous_day != current_day:
                temp_wallet = wallet
                if current_position:
                    if current_position['side'] == "LONG":
                        close_price = row['open']
                        trade_result = (close_price - current_position['price']) / current_position['price']
                        close_size = current_position['size'] + current_position['size']  * trade_result
                        fee = close_size * taker_fee
                        temp_wallet += close_size - current_position['size'] - fee
                    elif current_position['side'] == "SHORT":
                        close_price = row['open']
                        trade_result = (current_position['price'] - close_price) / current_position['price']
                        close_size = current_position['size'] + current_position['size']  * trade_result
                        fee = close_size * taker_fee
                        temp_wallet += close_size - current_position['size'] - fee
                    
                days.append({
                    "day":str(index.year)+"-"+str(index.month)+"-"+str(index.day),
                    "wallet":temp_wallet,
                    "price":row['open'],
                    "long_exposition":0,
                    "short_exposition":0,
                    "risk":0,
                })
            previous_day = current_day 
            
            # -- Close LONG --
            # Close Market    
            if current_position and current_position['side'] == "LONG" and row['close_long']:
                close_price = row['close']
                trade_result = (close_price - current_position['price']) / current_position['price']
                close_size = current_position['size'] + current_position['size'] * trade_result
                fee = close_size * maker_fee
                wallet += close_size - current_position['size'] - fee
                trades.append({
                    "pair": self.pair,
                    "open_date": current_position['date'],
                    "close_date": index,
                    "position": current_position['side'],
                    "open_reason": current_position['reason'],
                    "close_reason": "Market",
                    "open_price": current_position['price'],
                    "close_price": close_price,
                    "open_fee": current_position['fee'],
                    "close_fee": fee,
                    "open_trade_size":current_position['size'],
                    "close_trade_size":close_size,
                    "wallet": wallet,
                })
                current_position = None
                
            # -- Close SHORT --
            if current_position and current_position['side'] == "SHORT" and row['close_short']:
                close_price = row['close']
                trade_result = (current_position['price'] - close_price) / current_position['price']
                close_size = current_position['size'] + current_position['size'] * trade_result
                fee = close_size * taker_fee
                wallet += close_size - current_position['size'] - fee
                trades.append({
                    "pair": self.pair,
                    "open_date": current_position['date'],
                    "close_date": index,
                    "position": current_position['side'],
                    "open_reason": current_position['reason'],
                    "close_reason": "Market",
                    "open_price": current_position['price'],
                    "close_price": close_price,
                    "open_fee": current_position['fee'],
                    "close_fee": fee,
                    "open_trade_size":current_position['size'],
                    "close_trade_size":close_size,
                    "wallet": wallet,
                })
                current_position = None
                    
            # -- Check for opening position --
            # -- Open LONG --
            if current_position is None and row['open_long']:
                open_price = row["close"]
                pos_size = (params["size"] * wallet * leverage)
                fee = pos_size * taker_fee
                pos_size -= fee
                wallet -= fee
                current_position = {
                    "size": pos_size,
                    "date": index,
                    "price": open_price,
                    "fee":fee,
                    "reason": "Market",
                    "side": "LONG",
                }
            # -- Open SHORT --
            if current_position is None and row['open_short']:
                open_price = row["close"]
                pos_size = (params["size"] * wallet * leverage)
                fee = pos_size * taker_fee
                pos_size -= fee
                wallet -= fee
                current_position = {
                    "size": pos_size,
                    "date": index,
                    "price": open_price,
                    "fee":fee,
                    "reason": "Market",
                    "side": "SHORT",
                }             
                        
        df_days = pd.DataFrame(days)
        df_days['day'] = pd.to_datetime(df_days['day'])
        df_days = df_days.set_index(df_days['day'])

        df_trades = pd.DataFrame(trades)
        df_trades['open_date'] = pd.to_datetime(df_trades['open_date'])
        df_trades = df_trades.set_index(df_trades['open_date'])   
        
        return get_metrics(df_trades, df_days) | {
            "wallet": wallet,
            "trades": df_trades,
            "days": df_days
        }      

In [ ]:
params = {
    "fast_ma": 10,
    "slow_ma": 100,
    "signal": 20,
    "size": 1,
}

pair_list = list(params.keys())
exchange_name = "binance"
tf = '4h'
pair = "ETH/USDT"

exchange = ExchangeDataManager(
    exchange_name=exchange_name, 
    path_download="../database/exchanges"
)

df = exchange.load_data(pair, tf)

print("Data load 100%")

In [ ]:
strat = Strategy(
    pair=pair,
    df=df,
    type=["long"],
    params=params,
)

strat.populate_indicators()
strat.populate_buy_sell()
bt_result = strat.run_backtest(initial_wallet=1000, leverage=1)

df_trades, df_days = backtest_analysis(
    trades=bt_result['trades'], 
    days=bt_result['days'],
    general_info=True,
    trades_info=True,
    days_info=True,
    long_short_info=True,
    entry_exit_info=True,
    exposition_info=True,
    pair_info=True,
    indepedant_trade=True
)

In [ ]:
# df_trades.to_excel("trades.xlsx")
df_trades

In [ ]:
plot_equity_vs_asset(df_days=df_days.loc[:])

In [ ]:
plot_bar_by_month(df_days=df_days)

In [ ]:
df_trades.sort_values(by=["trade_result_pct"])